In [1]:
import requests, json
from pprint import pprint
import pandas as pd
pd.set_option('display.max_columns', None)
from tqdm.auto import tqdm
tqdm.pandas()

/Users/rachelchen/opt/anaconda3/lib/python3.8/site-packages/tqdm/std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
# base url for all FPL API endpoints
base_url = 'https://fantasy.premierleague.com/api/'

# get data from bootstrap-static endpoint
r = requests.get(base_url+'bootstrap-static/').json()

In [3]:
def get_season_history(player_id):
    '''get all past season info for a given player_id'''
    
    # send GET request to
    # https://fantasy.premierleague.com/api/element-summary/{PID}/
    r = requests.get(
            base_url + 'element-summary/' + str(player_id) + '/'
    ).json()
    
    # extract 'history_past' data from response into dataframe
    df = pd.json_normalize(r['history_past'])
    df['player_id'] = player_id
    
    return df

In [4]:
# create players dataframe
players = pd.json_normalize(r['elements'])
# create teams dataframe
teams = pd.json_normalize(r['teams'])
# get position information from 'element_types' field
positions = pd.json_normalize(r['element_types'])

In [5]:
players.head(1)

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,id,in_dreamteam,news,news_added,now_cost,photo,points_per_game,second_name,selected_by_percent,special,squad_number,status,team,team_code,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text
0,100.0,100.0,80201,0,0,-5,5,0,1,0.0,1.0,0,Bernd,0.0,1,False,,2022-02-11T08:00:15.144286Z,45,80201.jpg,1.3,Leno,0.8,False,None,a,1,3,4,64415,71,189600,106,0.0,0.9,Leno,270,0,0,0,9,0,0,0,0,0,9,0,48,79.0,0.0,0.0,7.9,367,29,600,69,581,60,425,30,NaN,,NaN,,NaN,


In [6]:
teams.head(1)

,code,draw,form,id,loss,name,played,points,position,short_name,strength,team_division,unavailable,win,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id
0,3,0,None,1,0,Arsenal,0,0,0,ARS,4,None,False,0,1210,1270,1150,1210,1190,1220,1


In [7]:
positions.head(1)

,id,plural_name,plural_name_short,singular_name,singular_name_short,squad_select,squad_min_play,squad_max_play,ui_shirt_specific,sub_positions_locked,element_count
0,1,Goalkeepers,GKP,Goalkeeper,GKP,2,1,1,True,[12],83


In [8]:
# select columns of interest from players df
players = players[
    ['id', 'first_name', 'second_name', 'web_name', 'team',
     'element_type']
]

# join team name
df = players.merge(
    teams[['id', 'name']],
    left_on='team',
    right_on='id',
    suffixes=['_player', None]
).drop(['team', 'id'], axis=1)

# join player positions
df = df.merge(
    positions[['id', 'singular_name']],
    left_on='element_type',
    right_on='id'
).drop(['element_type', 'id'], axis=1)

players.head()

,id,first_name,second_name,web_name,team,element_type
0,1,Bernd,Leno,Leno,1,1
1,2,Rúnar Alex,Rúnarsson,Rúnarsson,1,1
2,3,Willian,Borges Da Silva,Willian,1,3
3,4,Pierre-Emerick,Aubameyang,Aubameyang,1,4
4,5,Cédric,Soares,Cédric,1,2


In [9]:
# get gameweek histories for each player
points = df['id_player'].progress_apply(get_season_history)

# combine results into single dataframe
points = pd.concat(df for df in points)

In [10]:
points.head(1)

,season_name,element_code,start_cost,end_cost,total_points,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,player_id
0,2018/19,80201.0,50.0,49.0,106.0,2835.0,0.0,0.0,6.0,42.0,0.0,0.0,0.0,0.0,0.0,105.0,5.0,568.0,807.2,0.0,0.0,80.5,1


In [11]:
# join web_name
df = df[['id_player', 'first_name','second_name', 'name', 'singular_name']].merge(
    points,
    left_on='id_player',
    right_on='player_id'
)

In [14]:
df21_detail = df[df['season_name']=='2020/21'].sort_values(by=['total_points'], ascending = False)
df21_detail.head()

,id_player,first_name,second_name,name,singular_name,season_name,element_code,start_cost,end_cost,total_points,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,player_id
883,277,Bruno Miguel,Borges Fernandes,Man Utd,Midfielder,2020/21,141746.0,105.0,113.0,244.0,3101.0,18.0,14.0,13.0,36.0,0.0,0.0,1.0,6.0,0.0,0.0,36.0,870.0,1292.6,1414.9,1253.0,396.2,277
1433,357,Harry,Kane,Spurs,Forward,2020/21,78830.0,105.0,119.0,242.0,3083.0,23.0,14.0,12.0,39.0,0.0,0.0,0.0,1.0,0.0,0.0,40.0,880.0,1318.2,659.1,1585.0,355.9,357
774,233,Mohamed,Salah,Liverpool,Midfielder,2020/21,118748.0,120.0,129.0,231.0,3077.0,22.0,6.0,11.0,41.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0,657.0,1056.0,825.7,1980.0,385.8,233
1038,359,Heung-Min,Son,Spurs,Midfielder,2020/21,85971.0,90.0,96.0,228.0,3119.0,17.0,11.0,13.0,36.0,0.0,0.0,0.0,0.0,0.0,0.0,26.0,777.0,1052.2,1049.9,1046.0,315.2,359
1340,189,Patrick,Bamford,Leeds,Forward,2020/21,106617.0,55.0,66.0,194.0,3052.0,17.0,11.0,10.0,50.0,0.0,0.0,0.0,3.0,0.0,0.0,26.0,631.0,867.2,371.0,1512.0,274.6,189


In [15]:
df21_project = df21_detail[['id_player','first_name','second_name', 'name', 'singular_name', 'end_cost', 'total_points']]
df21_project.head()

,id_player,first_name,second_name,name,singular_name,end_cost,total_points
883,277,Bruno Miguel,Borges Fernandes,Man Utd,Midfielder,113.0,244.0
1433,357,Harry,Kane,Spurs,Forward,119.0,242.0
774,233,Mohamed,Salah,Liverpool,Midfielder,129.0,231.0
1038,359,Heung-Min,Son,Spurs,Midfielder,96.0,228.0
1340,189,Patrick,Bamford,Leeds,Forward,66.0,194.0


In [16]:
df21_detail.to_csv('df21_detail.csv', index = False)
df21_project.to_csv('df21_project.csv', index = False)